In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statistics

In [101]:
sample_data = pd.read_excel('./assets/sample_data_1.xlsx', sheet_name="X bar and R chart")
# sample_data = pd.read_excel('./assets/sample_data_1.xlsx', sheet_name="C chart")
sample_data.info()
sample_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Sample  15 non-null     int64  
 1   1       15 non-null     float64
 2   2       15 non-null     float64
 3   3       15 non-null     float64
 4   4       15 non-null     float64
 5   5       15 non-null     float64
dtypes: float64(5), int64(1)
memory usage: 848.0 bytes


,Sample,1,2,3,4,5
0,1,18.06,18.04,18.00,18.02,18.04
1,2,18.06,18.02,18.08,18.00,18.00
2,3,18.02,18.04,18.04,18.06,18.02
3,4,18.06,18.02,18.06,18.00,18.02
4,5,18.08,18.04,18.06,18.02,18.06


In [104]:
## is data discrete or continuous?
i = 0
num_col = len(sample_data.columns) - 1
flag_conti = True
while i < num_col:
    if (sample_data[i+1]%1 == 0).all():
        flag_conti = False
    i+=1
if flag_conti:
    print("Data is continuous")
    ## is there more than one data per subgroup?
    flag_subgroup = False
    num_col = len(sample_data.columns) - 1
    if num_col > 1 :
        flag_subgroup = True
        if num_col > 10:
            print("Data will use 'X bar & S chart'")
        else:
            print("Data will use 'X bar & R chart'")
    else:
        print("Data will use 'XmR chart'")
else:
    print("Data is discrete")
    ## does data follow Poisson distribution or Binomial 
    flag_poisson = False
    mean = round(sample_data[1].mean(),0)
    var = round((sample_data[1].var()),0)
    if(mean == var):
        flag_poisson = True
        print("Data will use either 'C chart' or 'U chart'")
    else:
        print("Data will use 'P chart'")
    ## print("Data is discrete and will use 'P chart' or 'C chart' or 'U chart' or 'G chart'")

Data is continuous
Data will use 'X bar & R chart'


In [5]:
## now that we know what chart to use for what type of data, let's create the respective charts

In [ ]:
## for x bar & r chart

x = np.array(sample_data.iloc[:,1:].to_numpy())  
## Define list variable for groups means
x_bar = []
## Define list variable for groups ranges
r = [] 
## Get and append groups means and ranges
for group in x:
    x_bar.append(round(group.mean(),3))
    r.append(round(group.max() - group.min(),3))
## Plot x-bar and R charts
fig, axs = plt.subplots(2, figsize=(10,10))

## x-bar chart
# axs[0].plot(x_bar, linestyle='-', marker='o', markersize=10, color='black', linewidth=2)
axs[0].plot(x_bar, linestyle='-', marker='o', color='black')
axs[0].axhline((statistics.mean(x_bar)+0.577*statistics.mean(r)), color='red', linestyle='dashed', label='UCL')
axs[0].axhline((statistics.mean(x_bar)-0.577*statistics.mean(r)), color='green', linestyle='dashed', label='LCL')
axs[0].axhline((statistics.mean(x_bar)), color='blue')
axs[0].set_title('X-bar Chart')
axs[0].set(xlabel='Sample', ylabel='Range')
axs[0].legend(fancybox=True, framealpha=1, shadow=True,frameon=True, borderpad=1)

## R chart
axs[1].plot(r, linestyle='-', marker='o', color='black')
axs[1].axhline((2.574*statistics.mean(r)), color='red', linestyle='dashed', label='UCL')
axs[1].axhline((0*statistics.mean(r)), color='green', linestyle='dashed', label='LCL')
axs[1].axhline((statistics.mean(r)), color='blue')
axs[1].set_ylim(bottom=0)
axs[1].set_title('R Chart')
axs[1].set(xlabel='Sample', ylabel='Range')
axs[1].legend(fancybox=True, framealpha=1, shadow=True,frameon=True, borderpad=1)

fig.tight_layout(pad=4)

In [ ]:
## x bar and S chart

x = np.array(sample_data.iloc[:,1:].to_numpy())  
## Define list variable for groups means
x_bar = []
## Define list variable for groups ranges
s = [] 
## Get and append groups means and ranges
for group in x:
    x_bar.append(round(group.mean(),3))
    s.append(round(group.max() - group.min(),3))
## Plot x-bar and S charts
fig, axs = plt.subplots(2, figsize=(10,10))

## x-bar chart
# axs[0].plot(x_bar, linestyle='-', marker='o', markersize=10, color='black', linewidth=2)
axs[0].plot(x_bar, linestyle='-', marker='o', color='black')
axs[0].axhline((statistics.mean(x_bar)+0.577*statistics.mean(r)), color='red', linestyle='dashed', label='UCL')
axs[0].axhline((statistics.mean(x_bar)-0.577*statistics.mean(r)), color='green', linestyle='dashed', label='LCL')
axs[0].axhline((statistics.mean(x_bar)), color='blue')
axs[0].set_title('X-bar Chart')
axs[0].set(xlabel='Sample', ylabel='Range')
axs[0].legend(fancybox=True, framealpha=1, shadow=True,frameon=True, borderpad=1)
## S chart
axs[1].plot(s, linestyle='-', marker='o', color='black')
axs[1].axhline((1.649*statistics.mean(r)), color='red', linestyle='dashed', label='UCL')
axs[1].axhline((0.321*statistics.mean(r)), color='green', linestyle='dashed', label='LCL')
axs[1].axhline((statistics.mean(r)), color='blue')
axs[1].set_ylim(bottom=0)
axs[1].set_title('S Chart')
axs[1].set(xlabel='Sample', ylabel='Range')
axs[1].legend(fancybox=True, framealpha=1, shadow=True,frameon=True, borderpad=1)

fig.tight_layout(pad=4)